In [1]:
import weka.core.jvm as jvm
import os
import traceback
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.core.dataset import Instances
from weka.timeseries import TSEvaluation, TSEvalModule, WekaForecaster,Periodicity
from weka.classifiers import Classifier
from weka.core.classes import serialization_write, serialization_read
jvm.start(packages=True)

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['C:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'C:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'C:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\javabridge\\jars\\cpython.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\arpack_combined.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\core.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\mtj.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxH

In [2]:
# load a dataset
from weka.core.converters import Loader
loader = Loader(classname="weka.core.converters.ArffLoader")
airline_data = loader.load_file("data/BTC-all.arff")
# airline_data.class_is_last()

In [3]:
# available evaluation modules
print("Evaluation modules")
modules = TSEvalModule.module_list()
print("Available modules")
for module in modules:
    print("-" + str(module))
print("Loading module by name")
print(TSEvalModule.module("MAE"))

Evaluation modules


AssertionError: 

In [6]:
# evaluate forecaster
print("Evaluate forecaster")
forecaster = WekaForecaster()
forecaster.fields_to_forecast = ["Adj Close"]

# forecaster.base_forecaster = Classifier(classname="weka.classifiers.functions.LinearRegression")
# forecaster.base_forecaster = Classifier(classname="weka.classifiers.rules.ZeroR")
# forecaster.base_forecaster = Classifier(classname="weka.classifiers.timeseries.HoltWinters")
forecaster.base_forecaster = Classifier(classname="weka.classifiers.trees.RandomForest")
# forecaster.base_forecaster = Classifier(classname="weka.classifiers.functions.GaussianProcesses")



forecaster.tslag_maker.timestamp_field = "Date"
forecaster.tslag_maker.adjust_for_variance = False

# add overlay data
forecaster.overlay_fields="Open,High,Low,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD"
print(forecaster.is_using_overlay_data)
print("forecast with overlays")
print(forecaster.forecast_with_overlays)

forecaster.tslag_maker.max_lag=7
forecaster.tslag_maker.min_lag=1
forecaster.tslag_maker.periodicity= Periodicity(periodicity="DAILY")



# print(help(forecaster.tslag_maker.periodicity))
# periodicity=None

forecaster.tslag_maker.include_powers_of_time = True
forecaster.tslag_maker.include_timelag_products = True
forecaster.tslag_maker.remove_leading_instances_with_unknown_lag_values = True

# forecaster.tslag_maker.add_month_of_year = True
# forecaster.tslag_maker.add_quarter_of_year = True

forecaster.tslag_maker.add_day_of_week=True
forecaster.tslag_maker.add_weekend_indicator=True

print("algorithm name: " + str(forecaster.algorithm_name))
print("command-line: " + forecaster.to_commandline())
print("lag maker: " + forecaster.tslag_maker.to_commandline())

evaluation = TSEvaluation(airline_data, 0.1)
evaluation.evaluate_on_training_data = True
evaluation.evaluate_on_test_data = True
# evaluation.prime_window_size = forecaster.tslag_maker.max_lag
evaluation.prime_for_test_data_with_test_data = True
evaluation.rebuild_model_after_each_test_forecast_step = False
evaluation.forecast_future = True
evaluation.horizon = 1
evaluation.prime_window_size=7
evaluation.evaluation_modules = "MAE,RMSE"


evaluation.evaluate(forecaster)
print(evaluation)
if evaluation.evaluate_on_training_data or evaluation.evaluate_on_test_data:
    print(evaluation.summary())
# if evaluation.evaluate_on_training_data:
#     print("Predictions (training data): " + evaluation.predictions_for_training_data(1).summary)
# if evaluation.evaluate_on_test_data:
#     print("Predictions (test data): " + evaluation.predictions_for_test_data(1).summary)
#     preds = evaluation.predictions_for_test_data(1)
#     print("Counts for targets: " + str(preds.counts_for_targets()))
#     print("Errors for target 'Adj Close': " + str(preds.errors_for_target("Adj Close")))
#     print("Errors for all targets: " + str(preds.predictions_for_all_targets()))
# if evaluation.training_data is not None:
#     print("Future forecasts (training)\n" + evaluation.print_future_forecast_on_training_data(forecaster))
# if evaluation.test_data is not None:
#     print("Future forecasts (test)\n" + evaluation.print_future_forecast_on_test_data(forecaster))

# if evaluation.evaluate_on_training_data:
#     print(evaluation.print_predictions_for_training_data("Predictions (training)", "Adj Close", 1))

# if evaluation.evaluate_on_test_data:
#     print(evaluation.print_predictions_for_test_data("Predictions (test)", "Adj Close", 1))

Evaluate forecaster
Failed to instantiate weka.classifiers.timeseries.WekaForecaster: weka.classifiers.timeseries.WekaForecaster
Class 'weka.classifiers.timeseries.WekaForecaster' is available from package: timeseriesForecasting


Exception: Failed to instantiate forecaster 'weka.classifiers.timeseries.WekaForecaster' - is package 'timeseriesForecasting' installed and jvm started with package support?

In [ ]:
# build forecaster
print("Build/use forecaster")
airline_train, airline_test = airline_data.train_test_split(90.0)
forecaster = WekaForecaster()
forecaster.fields_to_forecast = ["Adj Close"]
forecaster.base_forecaster = Classifier(classname="weka.classifiers.functions.LinearRegression")
forecaster.fields_to_forecast = "Adj Close"
forecaster.build_forecaster(airline_train)
num_prime_instances = 10
airline_prime = Instances.copy_instances(airline_train, airline_train.num_instances - num_prime_instances, num_prime_instances)
forecaster.prime_forecaster(airline_prime)
num_future_forecasts = airline_test.num_instances
preds = forecaster.forecast(num_future_forecasts)
print("Actual,Predicted,Error")
for i in range(num_future_forecasts):
    actual = airline_test.get_instance(i).get_value(0)
    predicted = preds[i][0].predicted
    error = actual - predicted
    print("%f,%f,%f" % (actual, predicted, error))

Build/use forecaster
Actual,Predicted,Error
1660233600000.000000,0.327720,1660233599999.672363
1660320000000.000000,0.326721,1660319999999.673340
1660406400000.000000,0.330433,1660406399999.669678
1660492800000.000000,0.328319,1660492799999.671631
1660579200000.000000,0.328934,1660579199999.671143
1660665600000.000000,0.331202,1660665599999.668701
1660752000000.000000,0.329016,1660751999999.670898
1660838400000.000000,0.333525,1660838399999.666504
1660924800000.000000,0.335632,1660924799999.664307
1661011200000.000000,0.333012,1661011199999.666992
1661097600000.000000,0.335121,1661097599999.664795
1661184000000.000000,0.337067,1661183999999.662842
1661270400000.000000,0.337289,1661270399999.662598
1661356800000.000000,0.338498,1661356799999.661621
1661443200000.000000,0.339860,1661443199999.660156
1661529600000.000000,0.339998,1661529599999.659912
1661616000000.000000,0.341837,1661615999999.658203
1661702400000.000000,0.343007,1661702399999.656982
1661788800000.000000,0.343315,16617887

In [ ]:
# serialization (if supported)
print("Serialization")
model_file = "base.model"
if forecaster.base_model_has_serializer:
    forecaster.save_base_model(model_file)
    forecaster2 = WekaForecaster()
    forecaster2.load_base_model(model_file)
    print(forecaster2.to_commandline())
else:
    print("Base model has no serializer, falling back to generic serialization")
    serialization_write(model_file, forecaster.base_forecaster)
    cls = Classifier(jobject=serialization_read(model_file))
    print(cls.to_commandline())

Serialization
Base model has no serializer, falling back to generic serialization
weka.classifiers.functions.LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4


In [ ]:
# state management
print("State")
model_file = "state.ser"
if forecaster.uses_state:
    forecaster.serialize_state(model_file)
    forecaster2 = WekaForecaster()
    forecaster2.load_serialized_state(model_file)
    print(forecaster2.to_commandline())
else:
    print("Forecaster does not use state, falling back to generic serialization")
    serialization_write(model_file, forecaster)
    forecaster2 = WekaForecaster(jobject=serialization_read(model_file))
    print(forecaster2.to_commandline())

State
Forecaster does not use state, falling back to generic serialization
weka.classifiers.timeseries.WekaForecaster -F "Adj Close" -L 1 -M 12 -B 2 -C 2 -G ArtificialTimeIndex -conf 0 -P 0.95 -W "weka.classifiers.functions.LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4"


In [2]:
jvm.stop()